In [19]:
import torch
import torchtext
from torchtext import data
import torch.optim as optim
import torch.nn as nn
import numpy as np
import argparse
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import gradio as gr
from model_defs import baseline, CNN_module, CNN_module_unfreeze

In [20]:
glove = torchtext.vocab.GloVe(name="6B",dim=100) # embedding size = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [39]:
model_file_cnn_unf = "models/unf_model_40_5_40_3_16_0.0001_30_5_0.9085.pt"
model_file_cnn = "models/model_40_5_40_3_16_0.0001_30_0.9085.pt"
model_file_bs = "models/baseline_model.pt"

model_cnn = CNN_module(glove, 40, 5, 40, 3)
model_cnn_unf = CNN_module_unfreeze(glove, 40, 5, 40, 3)
model_bs = baseline(glove)

model_cnn.load_state_dict(torch.load(model_file_cnn))
model_cnn_unf.load_state_dict(torch.load(model_file_cnn_unf))
model_bs.load_state_dict(torch.load(model_file_bs))

<All keys matched successfully>

In [51]:
def predict(sentence):
    tokens = sentence.split()
    token_ints = [glove.stoi.get(tok, len(glove.stoi)-1) for tok in tokens]
    token_tensor = torch.LongTensor(token_ints).view(-1,1)
    # print(token_tensor)
    # Model CNN
    model_cnn.eval()
    output_cnn, fc_cnn = model_cnn(token_tensor)
    model_cnn_unf.eval()
    output_cnn_unf, fc_cnn_unf = model_cnn_unf(token_tensor)
    model_bs.eval()
    prob_bs = model_bs(token_tensor)
    output_bs = torch.sigmoid(prob_bs)
    sub_or_ob_cnn = "Subjective" if output_cnn.item() > 0.5 else "Objective"
    sub_or_ob_cnn_unf = "Subjective" if output_cnn_unf.item() > 0.5 else "Objective"
    sub_or_ob_bs = "Subjective" if output_bs.item() > 0.5 else "Objective"
    # print("CNN: ", sub_or_ob_cnn, output_cnn.item())
    # print("CNN Unfreeze: ", sub_or_ob_cnn_unf, output_cnn_unf.item())
    # print("Baseline: ", sub_or_ob_bs, output_bs.item())
    print("CNN: ", sub_or_ob_cnn, output_cnn.item())
    print("CNN Unfreeze: ", sub_or_ob_cnn_unf, output_cnn_unf.item())
    print("Baseline: ", sub_or_ob_bs, output_bs.item())
    return sub_or_ob_cnn, output_cnn.item(), sub_or_ob_cnn_unf, output_cnn_unf.item(), sub_or_ob_bs, output_bs.item()

In [52]:
predict("I think you are very good")

CNN:  Subjective 0.9999947547912598
CNN Unfreeze:  Subjective 0.9999958276748657
Baseline:  Subjective 0.9992876648902893


('Subjective',
 0.9999947547912598,
 'Subjective',
 0.9999958276748657,
 'Subjective',
 0.9992876648902893)

In [53]:
# Gradio interface such that user can input a sentence and get the prediction and probability
iface = gr.Interface(fn=predict, inputs="text", 
                     outputs=["text", "number", "text", "number", "text", "number"],
                     title="Subjective or Objective", 
                     description="Enter a sentence and the model will predict whether it is subjective or objective")
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7869
Running on public URL: https://ea58820964763ba049.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


CNN:  Objective 0.02511909231543541
CNN Unfreeze:  Objective 0.004596529062837362
Baseline:  Subjective 0.7646906971931458
CNN:  Subjective 0.9955385327339172
CNN Unfreeze:  Subjective 0.9935093522071838
Baseline:  Subjective 0.8956016302108765
CNN:  Subjective 0.8044186234474182
CNN Unfreeze:  Objective 0.3689664900302887
Baseline:  Subjective 0.8043393492698669
CNN:  Subjective 0.5382722616195679
CNN Unfreeze:  Objective 0.39309820532798767
Baseline:  Subjective 0.7889913320541382
